In [2]:
import polars as pl
import json

Create 2 datasets:

1. Definitions that ARE in the corpus
    This is used to evaluate the retrieval. Possibly divided by language?

2. Definitions that ARE NOT in the corpus
    This is used to evaluate the generation.


Evaluation metrics:
- BLEU
This is used to measure lexical similarity
- BertScore
This is used to measure semantic similarity

Possibly also:
- BLEURT
This is used to measure the quality of the generated definitions. Or maybe just use 

## Datasets preparation

In [3]:
defs = pl.read_csv("../data/definitions_corpus/definitions.csv")
definendums_list = (defs.select(
    pl.col('label')
    .str.replace('#', '')
    .str.replace(r'([a-zà-ÿ])([A-Z])', r'${1} ${2}', n=-1)  # Add space between lowercase and uppercase letters
    .str.to_lowercase()  # Convert to lowercase after splitting
)
)['label'].to_list()
definendums_list

['obstacles to trade',
 'injury',
 'adverse trade effects',
 'union industry',
 'union enterprise',
 'services',
 'ancillary services undertaking',
 'authorisation',
 'branch',
 'close links',
 'competent authority',
 'commodity and emission allowance dealer',
 'control',
 'compliance with the group capital test',
 'credit institution',
 'derivatives',
 'financial institution',
 'gender neutral remuneration policy',
 'group',
 'consolidated situation',
 'group supervisor',
 'home member state',
 'host member state',
 'initial capital',
 'investment firm',
 'investment firm group',
 'investment holding company',
 'investment services and activities',
 'management body',
 'management body in its supervisory function',
 'mixed financial holding company',
 'means aparent undertaking other than afinancial holding company',
 'senior management',
 'parent undertaking',
 'subsidiary',
 'systemic risk',
 'union parent investment firm',
 'union parent investment holding company',
 'union parent 

In [ ]:
{'CELEX_ID': '32014L0065', 'Term': 'National Competition Authority'} [('national competition authority', 100.0, 768)]
{'CELEX_ID': '32015L0412', 'Term': 'Organic Product'} [('organic product', 100.0, 8540)]
{'CELEX_ID': '32016R1628', 'Term': 'Electric Vehicle'} [('electric vehicle', 100.0, 1632)]
{'CELEX_ID': '32016R1630', 'Term': 'Hybrid Vehicle'} [('hybrid vehicle', 100.0, 5229)]
{'CELEX_ID': '32016L0685', 'Term': 'Traffic Data'} [('traffic data', 100.0, 10445)]

defs.filter(pl.col('label') == '#personalData')

id,definition_text,def_n,label,dataset,document_id,frbr_work,frbr_expression
i64,str,str,str,str,str,str,str
2425,"""personal data: means personal …","""#def_4""","""#personalData""","""EurLex""","""32018R1862.xml""","""/akn/eu/act/regulation/2018-11…","""/akn/eu/act/regulation/2018-11…"
3113,"""personal data: means personal …","""#def_12""","""#personalData""","""EurLex""","""32019L1024.xml""","""/akn/eu/act/directive/2019-06-…","""/akn/eu/act/directive/2019-06-…"
3807,"""personal data: means any infor…","""#def_13""","""#personalData""","""EurLex""","""32017R0218.xml""","""/akn/eu/act/regulation/2017-02…","""/akn/eu/act/regulation/2017-02…"
4644,"""personal data: means personal …","""#def_8""","""#personalData""","""EurLex""","""32019L0770.xml""","""/akn/eu/act/directive/2019-05-…","""/akn/eu/act/directive/2019-05-…"
7075,"""personal data: means personal …","""#def_11""","""#personalData""","""EurLex""","""32018R1860.xml""","""/akn/eu/act/regulation/2018-11…","""/akn/eu/act/regulation/2018-11…"
8473,"""personal data: means any infor…","""#def_h""","""#personalData""","""EurLex""","""32016R0794.xml""","""/akn/eu/act/regulation/2016-05…","""/akn/eu/act/regulation/2016-05…"
9161,"""personal data: means personal …","""#def_5""","""#personalData""","""EurLex""","""32018R1861.xml""","""/akn/eu/act/regulation/2018-11…","""/akn/eu/act/regulation/2018-11…"
9903,"""personal data: means personal …","""#def_3""","""#personalData""","""EurLex""","""32019R1240.xml""","""/akn/eu/act/regulation/2019-06…","""/akn/eu/act/regulation/2019-06…"


In [39]:
defs.filter(pl.col('label') == '#trafficData')['definition_text'].to_list()

['traffic data: means historic and real-time data on road traffic characteristics;']

In [30]:
from LegalDefAgent.src.existdb import existdb_handler

d = {"metadata":{"id":8473,"definendum_label":"#personalData","dataset":"EurLex","document_id":"32016R0794.xml","frbr_work":"/akn/eu/act/regulation/2016-05-11/794/!main","frbr_expression":"/akn/eu/act/regulation/2016-05-11/794/eng@/!main"},"definition_text":"personal data: means any information relating to a data subject;"}

def get_definition_timeline(definition):
    cons = existdb_handler.find_consolidated(definition['metadata'])
    if cons:
        earliest_entries = {}
        for item in cons:
            print(item)
            definition_text = item['definition'].strip()
            current_entry = earliest_entries.get(definition_text)
            
            if current_entry is None or item['date'] < current_entry['date']:
                earliest_entries[definition_text] = item
                
        ordered_definitions = sorted(earliest_entries.values(), key=lambda x: x['date'])
        for entry in ordered_definitions:
            entry['date'] = entry['date'].strftime('%Y-%m-%d')

        return ordered_definitions

    static = existdb_handler.extract_definition_from_exist(definition['metadata'])
    if static:
        for entry in static:
            entry['date'] = entry['date'].strftime('%Y-%m-%d')
        return static
    return None

get_definition_timeline(d)

{'date': datetime.datetime(2022, 6, 28, 0, 0), 'definition': '(h) " personal data " means any information relating to a data subject;'}


[{'date': '2022-06-28',
  'definition': '(h) " personal data " means any information relating to a data subject;'}]

### LEOS

In [4]:
leos_eval_dataset = pl.read_excel("./Evaluation_LEOS_annotato_def.xlsx")

In [5]:
leos_dict = leos_eval_dataset.select(["CELEX", "Definition"]).rename({"CELEX": "CELEX_ID", "Definition": "Term"}).to_dicts()
leos_dict

[{'CELEX_ID': '32016R0679', 'Term': 'Non-personal Data'},
 {'CELEX_ID': '32016R0680', 'Term': 'Connected device'},
 {'CELEX_ID': '32016R0681', 'Term': 'IoT device'},
 {'CELEX_ID': '32016R0682', 'Term': 'Open Data'},
 {'CELEX_ID': '32016R0683', 'Term': 'Satellite Data'},
 {'CELEX_ID': '32016R0684', 'Term': 'Cookie'},
 {'CELEX_ID': '32016R0685', 'Term': 'Synthetic Data'},
 {'CELEX_ID': '32016R0686', 'Term': 'International Data Transfer'},
 {'CELEX_ID': '32016R0687', 'Term': 'Data Sharing activity'},
 {'CELEX_ID': '32016R0688', 'Term': 'Data Buyer'},
 {'CELEX_ID': '32011L0083', 'Term': 'Connected device'},
 {'CELEX_ID': '32011L0084', 'Term': 'IoT device'},
 {'CELEX_ID': '32011L0085', 'Term': 'Dangerous Product'},
 {'CELEX_ID': '32011L0086', 'Term': 'Dangerous Product Importer'},
 {'CELEX_ID': '32011L0087', 'Term': 'Intermediary Platform'},
 {'CELEX_ID': '32011L0088', 'Term': 'Internet Platform'},
 {'CELEX_ID': '32011L0089', 'Term': 'Content Creator'},
 {'CELEX_ID': '32011L0090', 'Term': '

In [7]:
from rapidfuzz import process, fuzz

leos_retrieval = []
leos_generation = []

for el in leos_dict:
    x = process.extract(el['Term'].lower(), definendums_list, scorer=fuzz.token_sort_ratio, limit=1, score_cutoff=95)
    if x: 
        print(el, x)
        leos_retrieval.append(el)
    else:
        leos_generation.append(el)

{'CELEX_ID': '32014L0065', 'Term': 'National Competition Authority'} [('national competition authority', 100.0, 768)]
{'CELEX_ID': '32015L0412', 'Term': 'Organic Product'} [('organic product', 100.0, 8540)]
{'CELEX_ID': '32016R1628', 'Term': 'Electric Vehicle'} [('electric vehicle', 100.0, 1632)]
{'CELEX_ID': '32016R1630', 'Term': 'Hybrid Vehicle'} [('hybrid vehicle', 100.0, 5229)]
{'CELEX_ID': '32016L0685', 'Term': 'Traffic Data'} [('traffic data', 100.0, 10445)]


In [8]:
print(len(leos_retrieval), len(leos_generation))

5 85


### LexDrafter

In [9]:
lexdrafter = pl.read_json('./llama_definition_combined.json')

lexdrafter['celex_id', 'term', 'original_definition']

celex_id,term,original_definition
str,str,str
"""32019R2016""","""0-star compartment""","""'0-star compartment' means a f…"
"""32019R2019""","""0-star compartment""","""'0-star compartment' means a f…"
"""32019R2016""","""1-star compartment""","""'1-star compartment' means a f…"
"""32019R2019""","""1-star compartment""","""'1-star compartment' means a f…"
"""32019R2016""","""2-star compartment""","""'2-star compartment' means a f…"
…,…,…
"""32015R1189""","""woody biomass""","""'woody biomass' means biomass …"
"""32013R0347""","""works""","""'works' means the purchase, su…"
"""32013R0543""","""year-ahead forecast margin""","""'year-ahead forecast margin' m…"


In [10]:
lexdrafter_retrieval = []
lexdrafter_generation = []

for el in lexdrafter['term'].to_list():
    x = process.extract(el.lower(), definendums_list, scorer=fuzz.token_sort_ratio, limit=1, score_cutoff=95)
    if x: 
        print(el, x)
        lexdrafter_retrieval.append(el)
    else:
        lexdrafter_generation.append(el)

0-star compartment [('0-star compartment', 100.0, 3413)]
0-star compartment [('0-star compartment', 100.0, 3413)]
1-star compartment [('1-star compartment', 100.0, 3414)]
1-star compartment [('1-star compartment', 100.0, 3414)]
2-star compartment [('2-star compartment', 100.0, 3415)]
2-star compartment [('2-star compartment', 100.0, 3415)]
3-star compartment [('3-star compartment', 100.0, 3416)]
3-star compartment [('3-star compartment', 100.0, 3416)]
Agency [('agency', 100.0, 5025)]
Authorised representative [('authorised representative', 100.0, 333)]
Bohunice programme [('bohunice programme', 100.0, 4742)]
Community [('community', 100.0, 899)]
Contracting Party [('contracting party', 100.0, 10748)]
European standard [('european standard', 100.0, 1684)]
Harmonised standard [('harmonised standard', 100.0, 338)]
Hazardous waste [('hazardous waste', 100.0, 5297)]
Importer [('importer', 100.0, 334)]
Information or status display [('information or status display', 100.0, 3815)]
Investor [(

In [13]:
print(len(lexdrafter_retrieval), len(lexdrafter_generation))

607 596


In [15]:
from LegalDefAgent.src.tools.definition_search import retriever

retriever.invoke('year-ahead forecast margin')

Open /home/leo/Desktop/dhdk/Master thesis/project/data/vec_db/definitions_vectors.db failed, the file has been opened by another program
Start local milvus failed


ConnectionConfigException: <ConnectionConfigException: (code=1, message=Open local milvus failed)>

---

# Retrieval

---

# Generation

## Perform Evaluation

In [75]:
[{'CELEX_ID': '32014L0065', 'Term': 'National Competition Authority', "original_definition": },
 {'CELEX_ID': '32015L0412', 'Term': 'Organic Product'},
 {'CELEX_ID': '32016R1628', 'Term': 'Electric Vehicle'},
 {'CELEX_ID': '32016R1630', 'Term': 'Hybrid Vehicle'},
 {'CELEX_ID': '32016L0685', 'Term': 'Traffic Data'}]

[{'CELEX_ID': '32014L0065', 'Term': 'National Competition Authority'},
 {'CELEX_ID': '32015L0412', 'Term': 'Organic Product'},
 {'CELEX_ID': '32016R1628', 'Term': 'Electric Vehicle'},
 {'CELEX_ID': '32016R1630', 'Term': 'Hybrid Vehicle'},
 {'CELEX_ID': '32016L0685', 'Term': 'Traffic Data'}]

In [86]:
from LegalDefAgent.src.agents import definitions_agent_eval
import uuid


definendum = "soft drink"
model = "groq-llama-3.3-70b-versatile"

async def get_definition(definendum, model):
    inputs = {"messages": [("user", f"what's the definition of {definendum}?")]}
    configurable = {"configurable": {"user_id": "1", "thread_id": uuid.uuid4().hex, "model": model}}

    async for msg in definitions_agent_eval.astream(inputs, configurable, stream_mode="values"):
        chat = msg

    return chat

In [96]:
res['messages']

[HumanMessage(content="what's the definition of Open Data?", additional_kwargs={}, response_metadata={}, id='f9c3e372-0f15-4075-9518-a7dcfd3ba089'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_6qg5', 'function': {'arguments': '{"config": {"run_name": "definition_search", "tags": ["legal", "definition"]}, "definendum": "Open Data", "legislation": null, "question": "what\'s the definition of Open Data?", "date_filters": null}', 'name': 'definition_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-17e4bb43-325c-4717-8288-b9a3a63875f6-0', tool_calls=[{'name': 'definition_search', 'args': {'config': {'run_name': 'definition_search', 'tags': ['legal', 'definition']}, 'definendum': 'Open Data', 'legislation': None, 'question': "what's the definition of Open Data?", 'date_filters': None}, 'id': 'call_6qg5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1179, 'output_tokens': 62, 'total_tokens': 1241}),
 To

In [99]:
for el in leos_retrieval:
    print(el['Term'])
    res = await get_definition(el['Term'], model)
    print(res['messages'][-2].content)

National Competition Authority
{'most_relevant_definition': {'metadata': {'id': 5889, 'dataset': 'EurLex', 'celex_id': '32019L0001'}, 'timeline': [{'date': '2018-12-11', 'definition': '(1) "national competition authority" means an authority designated by a Member State pursuant to Article 35 of Regulation (EC) No 1/2003 as being responsible for the application of Articles 101 and 102'}], 'eurovocs': ['EU competition policy', 'cartel', 'competition', 'competition law', 'restriction on competition', 'sanction (EU)', 'single market']}}
Organic Product
{'most_relevant_definition': {'metadata': {'id': 8540, 'dataset': 'EurLex', 'celex_id': '32018R0848'}, 'timeline': [{'date': '2018-06-14', 'definition': '(2) "organic product" means a product resulting from organic production, other than a product produced during the conversion period referred to in Article 10. The products of hunting or fishing of wild animals are not considered as organic products;'}], 'eurovocs': ['agricultural product', 

## Calculate Metrics

In [ ]:
!pip install -q evaluate

In [ ]:
import json

model = ""

# Load the JSON data
with open(f'../evaluation/{model}_generation.json', 'r') as json_file:
    data = json.load(json_file)

In [ ]:
# Initialize lists to store predictions and references
predictions = []
references = []

# Iterate through each item in the JSON data
for item in data:
  celex_id = item["CELEX_ID"]
  if item["existing_record"] == ["NEW TERM"]:
    original_text = item['original_definition']
    term = item['term']
    new_term = f"'{term}'"
    original_text = original_text.replace(term, new_term)
    generated_text = item['generated_definition']

    if ((generated_text != "") and (generated_text != "NO JSON AS AN OUTPUT OBTAINED")):
      # Add the generated text to predictions list
      predictions.append(generated_text)

      # Add the original text to references list
      references_item = [original_text]  # References for this item
      references.append(references_item)


---

### BLEU

In [19]:
import evaluate
bleu = evaluate.load("bleu")


In [ ]:
# Compute the BLEU score
bleu4_results = bleu.compute(predictions=predictions, references=references, max_order=4)
print(f"BLEU-4: {bleu4_results['bleu']}")

In [ ]:
# Compute the BLEU score
bleu3_results = bleu.compute(predictions=predictions, references=references, max_order=3)
print(f"BLEU-3: {bleu3_results['bleu']}")

In [ ]:
# Compute the BLEU score
blue2_results = bleu.compute(predictions=predictions, references=references, max_order=2)
print(f"BLEU-2: {blue2_results['bleu']}")

In [ ]:
# Compute the BLEU score
blue1_results = bleu.compute(predictions=predictions, references=references, max_order=1)
print(f"BLEU-1: {blue1_results['bleu']}")

---

### BertScore

In [ ]:
import evaluate
bertscore = evaluate.load("bertscore")

In [ ]:
# bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

bertscore_results = bertscore.compute(predictions=predictions, references=references, model_type="nlpaueb/legal-bert-base-uncased")

In [ ]:
# Extract the F1 scores
f1_scores = bertscore_results['f1']

# Calculate the mean of F1 scores
mean_f1 = sum(f1_scores) / len(f1_scores)

print(f"BERTScore_F1: {bertscore_results['f1']}")
print("--------")
print(f"Mean BERTScore F1: {mean_f1}")

In [ ]:
# Extract the precision scores
precision_scores = bertscore_results['precision']

# Calculate the mean of precision scores
mean_precision = sum(precision_scores) / len(precision_scores)

print(f"BERTScore_precision: {bertscore_results['precision']}")
print("--------")
print(f"Mean BERTScore precision: {mean_precision}")

In [ ]:
# Extract the recall scores
recall_scores = bertscore_results['recall']

# Calculate the mean of recall scores
mean_recall = sum(recall_scores) / len(recall_scores)

print(f"BERTScore_recall: {bertscore_results['recall']}")
print("--------")
print(f"Mean BERTScore recall: {mean_recall}")

---

### BLEURT
>BLEURT is an evaluation metric for Natural Language Generation. It takes a pair of sentences as input, a reference and a candidate, and it returns a score that indicates to what extent the candidate is fluent and conveys the meaning of the reference.